In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Button

%matplotlib qt

In [5]:
file = '../IQ_Cap/03-17-2021_150003'
Dfile = '../IQ_Cap/Drone-03-17-2021_153631'
NumSamp = 1024

samples = np.fromfile(file + '.iq', np.complex64)
drone_samp = np.fromfile(Dfile + '.iq', np.complex64)

In [6]:
params = dict()
with open(file + '.txt') as f:
    split = [a.strip('\n').split(': ') for a in f.readlines()]
    for i in split:
        params[i[0]] = int(i[1])    
    f.close()

In [7]:
Dparams = dict()
with open(Dfile + '.txt') as f:
    split = [a.strip('\n').split(': ') for a in f.readlines()]
    for i in split:
        Dparams[i[0]] = int(i[1]) if 'Drone' not in i[0] else i[1]
    f.close()

In [8]:
params

{'LO': 2440500000,
 'BW': 56000000,
 'Sample Rate': 40000000,
 'Buffer Size': 1024}

In [9]:
Dparams

{'LO': 2440500000,
 'BW': 56000000,
 'Sample Rate': 40000000,
 'Buffer Size': 1024,
 'Drone BW': '10Mhz',
 'Drone Center Freq': '2.4405'}

In [10]:
samp_range = np.arange(0,NumSamp) #+ params['Buffer Size']*3

In [11]:
def FFT(samples, buf, samprate, lo):
    fft = np.fft.fftshift(np.fft.fft(samples)) / buf
    freqLabels = np.fft.fftshift(np.fft.fftfreq(buf, 1/samprate)) + lo
    fft_mag = np.log10(np.abs(fft))
    return fft_mag, freqLabels


In [12]:
freqData, freqLabels = FFT(samples[samp_range], NumSamp, params['Sample Rate'], params['LO'])
droneFreqData, droneFreqLabels = FFT(drone_samp[samp_range], NumSamp, Dparams['Sample Rate'], Dparams['LO'])

In [25]:
timeData = dict()
timeData['I'] = np.real(samples[samp_range])
timeData['Q'] = np.imag(samples[samp_range])

droneTimeData = dict()
droneTimeData['I'] = np.real(drone_samp[samp_range])
droneTimeData['Q'] = np.imag(drone_samp[samp_range])

fig = plt.figure(figsize=(15,10))
plt.subplot(2,2,1)
plt.title('Nominal Time Domain')
plt.xlabel('Samples')
plt.ylabel('Amplitude')
Nti, = plt.plot(timeData['I'], label='I')
Ntq, = plt.plot(timeData['Q'], label='Q')
plt.subplot(2,2,2)
plt.title('Drone Time Domain')
plt.xlabel('Samples')
plt.ylabel('Amplitude')
Dti, = plt.plot(droneTimeData['I'], label='I')
Dtq, = plt.plot(droneTimeData['Q'], label='Q')
plt.subplot(2,2,3)
plt.title('Nominal Frequency Domain')
plt.xlabel('Frequency')
plt.ylabel('Magnitude')
Nf, = plt.plot(freqLabels, freqData, label='Nominal')
plt.subplot(2,2,4)
plt.title('Drone Frequency Domain')
plt.xlabel('Frequency')
plt.ylabel('Magnitude')
Df, = plt.plot(droneFreqLabels, droneFreqData, label='Drone Present')

fig.text(.3,.95,f"LO: {params['LO']} | Buffer: {params['Buffer Size']} | Sample Rate: {params['Sample Rate']}", fontsize=12)
fig.text(.3,.92,f"RF Bandwidth: {params['BW']} | Drone Bandwidth: {Dparams['Drone BW']} | Drone Center Frequency: {Dparams['Drone Center Freq']}", fontsize=12)


text = fig.text(.85,.935,f"Samples: {samp_range[0]} to {params['Buffer Size']}", fontsize=10)

class Index():
    ind = 0

    def next(self, event):
        self.ind += NumSamp
        i = self.ind % len(samples)//NumSamp
        
        Ntupi = np.real(samples[samp_range + params['Buffer Size']*i])
        Ntupq = np.imag(samples[samp_range + params['Buffer Size']*i])
        
        Dtupi = np.real(drone_samp[samp_range + params['Buffer Size']*i])
        Dtupq = np.imag(drone_samp[samp_range + params['Buffer Size']*i])
        
        freqData, freqLabels = FFT(samples[samp_range + params['Buffer Size']*i], NumSamp, params['Sample Rate'], params['LO'])
        droneFreqData, droneFreqLabels = FFT(drone_samp[samp_range + params['Buffer Size']*i], NumSamp, Dparams['Sample Rate'], Dparams['LO'])
        
        Nti.set_ydata(Ntupi)
        Ntq.set_ydata(Ntupq)
        
        Dti.set_ydata(Ntupi)
        Dtq.set_ydata(Ntupq)

        Nf.set_xdata(freqLabels)
        Nf.set_ydata(freqData)
        Df.set_xdata(droneFreqLabels)
        Df.set_ydata(droneFreqData)
        
        
        #print("Next!")
        
        text.set_text(f"Samples: {samp_range[0] + params['Buffer Size']*i} to {samp_range[-1] + params['Buffer Size']*i}")
        
        plt.draw()

    def prev(self, event):
        self.ind -= NumSamp
        i = self.ind % len(samples)//NumSamp
        
        Ntupi = np.real(samples[samp_range + params['Buffer Size']*i])
        Ntupq = np.imag(samples[samp_range + params['Buffer Size']*i])
        
        Dtupi = np.real(drone_samp[samp_range + params['Buffer Size']*i])
        Dtupq = np.imag(drone_samp[samp_range + params['Buffer Size']*i])
        
        freqData, freqLabels = FFT(samples[samp_range + params['Buffer Size']*i], NumSamp, params['Sample Rate'], params['LO'])
        droneFreqData, droneFreqLabels = FFT(drone_samp[samp_range + params['Buffer Size']*i], NumSamp, Dparams['Sample Rate'], Dparams['LO'])
        
        Nti.set_ydata(Ntupi)
        Ntq.set_ydata(Ntupq)
        
        Dti.set_ydata(Ntupi)
        Dtq.set_ydata(Ntupq)

        Nf.set_xdata(freqLabels)
        Nf.set_ydata(freqData)
        Df.set_xdata(droneFreqLabels)
        Df.set_ydata(droneFreqData)
        
        #print("Prev!")
        
        text.set_text(f"Samples: {samp_range[0] + params['Buffer Size']*i} to {samp_range[-1] + params['Buffer Size']*i}")
        
        plt.draw()

callback = Index()
axprev = plt.axes([0.9, 0.43, 0.1, 0.075])
axnext = plt.axes([0.9, 0.53, 0.1, 0.075])
bnext = Button(axnext, 'Next')
bnext.on_clicked(callback.next)
bprev = Button(axprev, 'Previous')
bprev.on_clicked(callback.prev)

0